# HW1: Frame-Level Speech Recognition

In this homework, you will be working with MFCC data consisting of 15 features at each time step/frame. Your model should be able to recognize the phoneme occured in that frame.

# Libraries

In [1]:
# !pip install torchsummaryX wandb --quiet

In [2]:
import torch
import numpy as np
from torchsummaryX import summary
import sklearn
from sklearn.metrics import accuracy_score
import gc
import zipfile
import pandas as pd
from tqdm.auto import tqdm
import os
import datetime
import wandb
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device: ", device)

Device:  cuda


In [3]:
### If you are using colab, you can import google drive to save model checkpoints in a folder
# from google.colab import drive
# drive.mount('/content/drive')

In [4]:
### PHONEME LIST
PHONEMES = [
            'SIL',   'AA',    'AE',    'AH',    'AO',    'AW',    'AY',  
            'B',     'CH',    'D',     'DH',    'EH',    'ER',    'EY',
            'F',     'G',     'HH',    'IH',    'IY',    'JH',    'K',
            'L',     'M',     'N',     'NG',    'OW',    'OY',    'P',
            'R',     'S',     'SH',    'T',     'TH',    'UH',    'UW',
            'V',     'W',     'Y',     'Z',     'ZH',    '<sos>', '<eos>']

# Kaggle

This section contains code that helps you install kaggle's API, creating kaggle.json with you username and API key details. Make sure to input those in the given code to ensure you can download data from the competition successfully.

In [5]:
# !pip install --upgrade --force-reinstall --no-deps kaggle==1.5.8
# !mkdir /root/.kaggle

# with open("/root/.kaggle/kaggle.json", "w+") as f:
#     f.write('{"username":"","key":""}') 
#     # Put your kaggle username & key here

# !chmod 600 /root/.kaggle/kaggle.json

In [6]:
# # commands to download data from kaggle

# !kaggle competitions download -c 11-785-f22-hw1p2
# !mkdir '/content/data'

# !unzip -qo '11-785-f22-hw1p2.zip' -d '/content/data'

# Dataset

This section covers the dataset/dataloader class for speech data. You will have to spend time writing code to create this class successfully. We have given you a lot of comments guiding you on what code to write at each stage, from top to bottom of the class. Please try and take your time figuring this out, as it will immensely help in creating dataset/dataloader classes for future homeworks.

Before running the following cells, please take some time to analyse the structure of data. Try loading a single MFCC and its transcipt, print out the shapes and print out the values. Do the transcripts look like phonemes?

### Deviation

In [7]:
dev_dirs = os.listdir('data/dev-clean/transcript')
print(len(dev_dirs))

SIL_cnts = 0
for i in range(len(dev_dirs)):
    dev_labels = np.load(os.path.join('data/dev-clean/transcript', dev_dirs[i]))
    SIL_cnts += list(dev_labels).count('SIL')
    
print(SIL_cnts)

2703
396367


### Dataset Class

In [8]:
# Dataset class to load train and validation data

class AudioDataset(torch.utils.data.Dataset):

    def __init__(self, data_path, context, offset=0, partition= "train", limit=-1): # Feel free to add more arguments

        self.context = context
        self.offset = offset
        self.data_path = data_path

        if partition == 'train':
            self.mfcc_dir = os.path.join(self.data_path, partition+'-clean-100', 'mfcc') # TODO: MFCC directory - use partition to acces train/dev directories from kaggle data
            self.transcript_dir = os.path.join(self.data_path, partition+'-clean-100', 'transcript') # TODO: Transcripts directory - use partition to acces train/dev directories from kaggle data
        else:
            self.mfcc_dir = os.path.join(self.data_path, partition+'-clean', 'mfcc') # TODO: MFCC directory - use partition to acces train/dev directories from kaggle data
            self.transcript_dir = os.path.join(self.data_path, partition+'-clean', 'transcript') # TODO: Transcripts directory - use partition to acces train/dev directories from kaggle data
            
        mfcc_names = sorted(os.listdir(self.mfcc_dir)) # TODO: List files in sefl.mfcc_dir_dir using os.listdir in sorted order, optionally subset using limit to slice the number of files you load
        transcript_names = sorted(os.listdir(self.transcript_dir)) # TODO: List files in self.transcript_dir using os.listdir in sorted order, optionally subset using limit to slice the number of files you load

        assert len(mfcc_names) == len(transcript_names) # Making sure that we have the same no. of mfcc and transcripts

        self.mfccs, self.transcripts = [], []

        # TODO:
        # Iterate through mfccs and transcripts
        for i in range(0, len(mfcc_names)):
        #   Load a single mfcc
            mfcc = np.load(os.path.join(self.mfcc_dir, mfcc_names[i]))
        #   Optionally do Cepstral Normalization of mfcc
        #   Load the corresponding transcript
            transcript = np.load(os.path.join(self.transcript_dir, transcript_names[i]))[1:-1] # Remove [SOS] and [EOS] from the transcript (Is there an efficient way to do this 
            # without traversing through the transcript?)
        #   Append each mfcc to self.mfcc, transcript to self.transcript
            self.mfccs.append(mfcc)
            self.transcripts.append(transcript)

        

        # NOTE:
        # Each mfcc is of shape T1 x 15, T2 x 15, ...
        # Each transcript is of shape (T1+2) x 15, (T2+2) x 15 before removing [SOS] and [EOS]

        # TODO: Concatenate all mfccs in self.mfccs such that the final shape is T x 15 (Where T = T1 + T2 + ...) 
        self.mfccs = np.concatenate(self.mfccs)
        print(self.mfccs.shape)

        # TODO: Concatenate all transcripts in self.transcripts such that the final shape is (T,) meaning, each time step has one phoneme output
        self.transcripts = np.concatenate(self.transcripts)
        print(self.transcripts.shape)
        # Hint: Use numpy to concatenate

        # Take some time to think about what we have done. self.mfcc is an array of the format (Frames x Features). Our goal is to recognize phonemes of each frame
        # From hw0, you will be knowing what context is. We can introduce context by padding zeros on top and bottom of self.mfcc
        self.mfccs = np.pad(self.mfccs, ((config['context'], config['context']), (0,0)), mode='constant', constant_values=0) # TODO
        print(self.mfccs.shape)

        # These are the available phonemes in the transcript
        self.phonemes = [
            'SIL',   'AA',    'AE',    'AH',    'AO',    'AW',    'AY',  
            'B',     'CH',    'D',     'DH',    'EH',    'ER',    'EY',
            'F',     'G',     'HH',    'IH',    'IY',    'JH',    'K',
            'L',     'M',     'N',     'NG',    'OW',    'OY',    'P',
            'R',     'S',     'SH',    'T',     'TH',    'UH',    'UW',
            'V',     'W',     'Y',     'Z',     'ZH',    '<sos>', '<eos>']
        # But the neural network cannot predict strings as such. Instead we map these phonemes to integers

        self.transcripts = np.array([self.phonemes.index(i) for i in self.transcripts]) # TODO: Map the phonemes to their corresponding list indexes in self.phonemes
        # Now, if an element in self.transcript is 0, it means that it is 'SIL' (as per the above example)
        
        # Length of the dataset is now the length of concatenated mfccs/transcripts
        self.length = len(self.mfccs) - 2*self.context

    def __len__(self):
        return self.length

    def __getitem__(self, ind):
        
        frames = self.mfccs[ind:ind + 2*self.context + 1, :] # TODO: Based on context and offset, return a frame at given index with context frames to the left, and right.
        # After slicing, you get an array of shape 2*context+1 x 15. But our MLP needs 1d data and not 2d.
        frames = np.ndarray.flatten(frames) # TODO: Flatten to get 1d data

        frames = torch.FloatTensor(frames) # Convert to tensors
        phoneme = torch.tensor(self.transcripts[ind])  

        return frames, phoneme

In [9]:
class AudioTestDataset(torch.utils.data.Dataset):

    # TODO: Create a test dataset class similar to the previous class but you dont have transcripts for this
    # Imp: Read the mfccs in sorted order, do NOT shuffle the data here or in your dataloader.

    def __init__(self, data_path, context, offset=0, limit=-1): # Feel free to add more arguments

        self.context = context
        self.offset = offset
        self.data_path = data_path

        self.mfcc_dir = os.path.join(self.data_path, 'test-clean', 'mfcc') # TODO: MFCC directory - use partition to acces train/dev directories from kaggle data
            
        mfcc_names = sorted(os.listdir(self.mfcc_dir)) # TODO: List files in sefl.mfcc_dir_dir using os.listdir in sorted order, optionally subset using limit to slice the number of files you load

        self.mfccs = []

        # TODO:
        # Iterate through mfccs and transcripts
        for i in range(0, len(mfcc_names)):
        #   Load a single mfcc
            mfcc = np.load(os.path.join(self.mfcc_dir, mfcc_names[i]))
        #   Optionally do Cepstral Normalization of mfcc
        #   Append each mfcc to self.mfcc, transcript to self.transcript
            self.mfccs.append(mfcc)
        

        # NOTE:
        # Each mfcc is of shape T1 x 15, T2 x 15, ...
        # Each transcript is of shape (T1+2) x 15, (T2+2) x 15 before removing [SOS] and [EOS]

        # TODO: Concatenate all mfccs in self.mfccs such that the final shape is T x 15 (Where T = T1 + T2 + ...) 
        self.mfccs = np.concatenate(self.mfccs)

        # Take some time to think about what we have done. self.mfcc is an array of the format (Frames x Features). Our goal is to recognize phonemes of each frame
        # From hw0, you will be knowing what context is. We can introduce context by padding zeros on top and bottom of self.mfcc
        self.mfccs = np.pad(self.mfccs, ((self.context, self.context), (0,0)), mode='constant', constant_values=0) # TODO

        # These are the available phonemes in the transcript
        self.phonemes = [
            'SIL',   'AA',    'AE',    'AH',    'AO',    'AW',    'AY',  
            'B',     'CH',    'D',     'DH',    'EH',    'ER',    'EY',
            'F',     'G',     'HH',    'IH',    'IY',    'JH',    'K',
            'L',     'M',     'N',     'NG',    'OW',    'OY',    'P',
            'R',     'S',     'SH',    'T',     'TH',    'UH',    'UW',
            'V',     'W',     'Y',     'Z',     'ZH',    '<sos>', '<eos>']
        # But the neural network cannot predict strings as such. Instead we map these phonemes to integers

        
        # Length of the dataset is now the length of concatenated mfccs/transcripts
        self.length = len(self.mfccs) - 2*self.context

    def __len__(self):
        return self.length

    def __getitem__(self, ind):
        
        frames = self.mfccs[ind:ind + 2*self.context + 1, :] # TODO: Based on context and offset, return a frame at given index with context frames to the left, and right.
        # After slicing, you get an array of shape 2*context+1 x 15. But our MLP needs 1d data and not 2d.
        frames = np.ndarray.flatten(frames) # TODO: Flatten to get 1d data

        frames = torch.FloatTensor(frames) # Convert to tensors

        return frames

# Parameters Configuration

Storing your parameters and hyperparameters in a single configuration dictionary makes it easier to keep track of them during each experiment. It can also be used with weights and biases to log your parameters for each experiment and keep track of them across multiple experiments. 

#### Model hyperparameter settings

I changed the model hyperparameters as following config dictionary. While training the model, the CosineAnnealing scheduler and AdamW optimizer is used. Also, the weight_decay value is set for the regularization since the size of model is big enough. 

In [10]:
config = {
    'epochs': 50,
    'batch_size' : 2048,
    'context' : 20,
    'learning_rate' : 0.001,
    'architecture' : 'cylinder',
    'dropout' : 0.2,
    'scheduler' : 'Cosine',
    'T_max' : 10,
    'weight_decay' : 1e-6,
    'optimizer' : 'AdamW'
    # Add more as you need them - e.g dropout values, weight decay, scheduler parameters
}

# Create Datasets

In [11]:
train_data = AudioDataset('data', context=config['context'], partition='train') #TODO: Create a dataset object using the AudioDataset class for the training data 

val_data = AudioDataset('data', context=config['context'], partition='dev') # TODO: Create a dataset object using the AudioDataset class for the validation data 

test_data = AudioTestDataset('data', context=config['context']) # TODO: Create a dataset object using the AudioTestDataset class for the test data 

(36191134, 15)
(36191134,)
(36191174, 15)
(1937496, 15)
(1937496,)
(1937536, 15)


In [12]:
# Define dataloaders for train, val and test datasets
# Dataloaders will yield a batch of frames and phonemes of given batch_size at every iteration
train_loader = torch.utils.data.DataLoader(train_data, num_workers= 4,
                                           batch_size=config['batch_size'], pin_memory= True,
                                           shuffle= True)

val_loader = torch.utils.data.DataLoader(val_data, num_workers= 2,
                                         batch_size=config['batch_size'], pin_memory= True,
                                         shuffle= False)

test_loader = torch.utils.data.DataLoader(test_data, num_workers= 2, 
                                          batch_size=config['batch_size'], pin_memory= True, 
                                          shuffle= False)


print("Batch size: ", config['batch_size'])
print("Context: ", config['context'])
print("Input size: ", (2*config['context']+1)*15)
print("Output symbols: ", len(PHONEMES))

print("Train dataset samples = {}, batches = {}".format(train_data.__len__(), len(train_loader)))
print("Validation dataset samples = {}, batches = {}".format(val_data.__len__(), len(val_loader)))
print("Test dataset samples = {}, batches = {}".format(test_data.__len__(), len(test_loader)))

Batch size:  2048
Context:  20
Input size:  615
Output symbols:  42
Train dataset samples = 36191134, batches = 17672
Validation dataset samples = 1937496, batches = 947
Test dataset samples = 1943253, batches = 949


In [13]:
# Testing code to check if your data loaders are working
for i, data in enumerate(train_loader):
    frames, phoneme = data
    print(frames.shape, phoneme.shape)
    break

torch.Size([2048, 615]) torch.Size([2048])


In [14]:
# Testing code to check if your data loaders are working
for i, data in enumerate(test_loader):
    frames = data
    print(frames.shape)
    break

torch.Size([2048, 615])


# Network Architecture


This section defines your network architecture for the homework. We have given you a sample architecture that can easily clear the very low cutoff for the early submission deadline.

#### The description about the architecture
The MLP architecture I made has 5 hidden layers with 2048 dimensions. The shape of the architecture is almost cylinder except the last hidden and output layer. 
We can run and train this model by running all the cells in the order. 

In [15]:
# This architecture will make you cross the very low cutoff
# However, you need to run a lot of experiments to cross the medium or high cutoff
class Network(torch.nn.Module):

    def __init__(self, context, dropout):

        super(Network, self).__init__()

        input_size = (2*context + 1) * 15 #Why is this the case? #375
        output_size = 40 #Why?

        self.model = torch.nn.Sequential(
            torch.nn.Linear(input_size, 2048),
            torch.nn.ReLU(),    
            torch.nn.Dropout(dropout),
            torch.nn.Linear(2048, 2048),
            torch.nn.BatchNorm1d(2048),
            torch.nn.ReLU(),
            torch.nn.Dropout(dropout),
            torch.nn.Linear(2048, 2048),
            torch.nn.BatchNorm1d(2048),
            torch.nn.ReLU(),
            torch.nn.Dropout(dropout),
            torch.nn.Linear(2048, 2048),
            torch.nn.BatchNorm1d(2048),
            torch.nn.ReLU(),
            torch.nn.Dropout(dropout),
            torch.nn.Linear(2048, 1024),
            torch.nn.ReLU(),
            torch.nn.Dropout(dropout),
            torch.nn.Linear(1024, 512),
            torch.nn.ReLU(),
            torch.nn.Linear(512, output_size)
        )      

    def forward(self, x):
        out = self.model(x)

        return out

# Define Model, Loss Function and Optimizer

Here we define the model, loss function, optimizer and optionally a learning rate scheduler. 

In [16]:
input_size = 15*(2*config['context'] + 1)
model = Network(config['context'], config['dropout']).to(device)
frames,phoneme = next(iter(train_loader))
# Check number of parameters of your network - Remember, you are limited to 20 million parameters for HW1 (including ensembles)
summary(model, frames.to(device))

                         Kernel Shape  Output Shape     Params  Mult-Adds
Layer                                                                    
0_model.Linear_0          [615, 2048]  [2048, 2048]  1.261568M   1.25952M
1_model.ReLU_1                      -  [2048, 2048]          -          -
2_model.Dropout_2                   -  [2048, 2048]          -          -
3_model.Linear_3         [2048, 2048]  [2048, 2048]  4.196352M  4.194304M
4_model.BatchNorm1d_4          [2048]  [2048, 2048]     4.096k     2.048k
5_model.ReLU_5                      -  [2048, 2048]          -          -
6_model.Dropout_6                   -  [2048, 2048]          -          -
7_model.Linear_7         [2048, 2048]  [2048, 2048]  4.196352M  4.194304M
8_model.BatchNorm1d_8          [2048]  [2048, 2048]     4.096k     2.048k
9_model.ReLU_9                      -  [2048, 2048]          -          -
10_model.Dropout_10                 -  [2048, 2048]          -          -
11_model.Linear_11       [2048, 2048] 

/home/jiin/anaconda3/envs/torch/lib/python3.9/site-packages/torchsummaryX/torchsummaryX.py:101: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_sum = df.sum()


,Kernel Shape,Output Shape,Params,Mult-Adds
Layer,,,,
0_model.Linear_0,"[615, 2048]","[2048, 2048]",1261568.0,1259520.0
1_model.ReLU_1,-,"[2048, 2048]",NaN,NaN
2_model.Dropout_2,-,"[2048, 2048]",NaN,NaN
3_model.Linear_3,"[2048, 2048]","[2048, 2048]",4196352.0,4194304.0
4_model.BatchNorm1d_4,[2048],"[2048, 2048]",4096.0,2048.0
5_model.ReLU_5,-,"[2048, 2048]",NaN,NaN
6_model.Dropout_6,-,"[2048, 2048]",NaN,NaN
7_model.Linear_7,"[2048, 2048]","[2048, 2048]",4196352.0,4194304.0
8_model.BatchNorm1d_8,[2048],"[2048, 2048]",4096.0,2048.0


In [17]:
criterion = torch.nn.CrossEntropyLoss() #Defining Loss function 
optimizer = torch.optim.AdamW(model.parameters(), lr=config['learning_rate'], weight_decay=config['weight_decay']) #Defining Optimizer
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=config['epochs'], eta_min=0)

# Recommended : Define Scheduler for Learning Rate, including but not limited to StepLR, MultiStepLR, CosineAnnealingLR, ReduceLROnPlateau, etc. 
# You can refer to Pytorch documentation for more information on how to use them.

# Training and Validation Functions

This section covers the training, and validation functions for each epoch of running your experiment with a given model architecture. The code has been provided to you, but we recommend going through the comments to understand the workflow to enable you to write these loops for future HWs.

In [18]:
torch.cuda.empty_cache()
gc.collect()

9

In [19]:
def train(model, optimizer, criterion, scheduler, dataloader):

    model.train()
    train_loss = 0.0 #Monitoring Loss
    
    for iter, (mfccs, phonemes) in tqdm(enumerate(dataloader)):

        ### Move Data to Device (Ideally GPU)
        mfccs = mfccs.to(device)
        phonemes = phonemes.to(device)

        ### Forward Propagation
        logits = model(mfccs)

        ### Loss Calculation
        loss = criterion(logits, phonemes)
        train_loss += loss.item()

        ### Initialize Gradients
        optimizer.zero_grad()

        ### Backward Propagation
        loss.backward()

        ### Gradient Descent
        optimizer.step()
    ### update scheduler
    scheduler.step()
        
    train_loss /= len(dataloader)
    return train_loss

In [20]:
def eval(model, dataloader):

    model.eval() # set model in evaluation mode
    val_loss = 0.0

    phone_true_list = []
    phone_pred_list = []

    for i, data in enumerate(dataloader):

        frames, phonemes = data
        ### Move data to device (ideally GPU)
        frames, phonemes = frames.to(device), phonemes.to(device) 

        with torch.inference_mode(): # makes sure that there are no gradients computed as we are not training the model now
            ### Forward Propagation
            logits = model(frames)
            loss = criterion(logits, phonemes)
            val_loss += loss.item()

        ### Get Predictions
        predicted_phonemes = torch.argmax(logits, dim=1)
        
        ### Store Pred and True Labels
        phone_pred_list.extend(predicted_phonemes.cpu().tolist())
        phone_true_list.extend(phonemes.cpu().tolist())
        
        # Do you think we need loss.backward() and optimizer.step() here?
    
        del frames, phonemes, logits
        torch.cuda.empty_cache()

    ### Calculate Accuracy
    accuracy = sklearn.metrics.accuracy_score(phone_pred_list, phone_true_list)
    val_loss /= len(dataloader)
    return accuracy*100, val_loss

# Weights and Biases Setup

This section is to enable logging metrics and files with Weights and Biases. Please refer to wandb documentationa and recitation 0 that covers the use of weights and biases for logging, hyperparameter tuning and monitoring your runs for your homeworks. Using this tool makes it very easy to show results when submitting your code and models for homeworks, and also extremely useful for study groups to organize and run ablations under a single team in wandb. 

We have written code for you to make use of it out of the box, so that you start using wandb for all your HWs from the beginning.

In [21]:
wandb.login(key=" ") #API Key is in your wandb account, under settings (wandb.ai/settings)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: jiin. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/jiin/.netrc


True

In [22]:
# Create your wandb run
run = wandb.init(
    name = "exp_structure_v2", ### Wandb creates random run names if you skip this field, we recommend you give useful names
    reinit=True, ### Allows reinitalizing runs when you re-run this cell
    project="hw1p2", ### Project should be created in your wandb account 
    config=config ### Wandb Config for your run
)

In [23]:
### Save your model architecture as a string with str(model) 
model_arch = str(model)

### Save it in a txt file 
arch_file = open("model_arch.txt", "w")
file_write = arch_file.write(model_arch)
arch_file.close()

### log it in your wandb run with wandb.save()
wandb.save('model_arch.txt')

['/media/usr/SSD/jiin/CMU/IDL/HW1/wandb/run-20220930_033518-rfq5ks30/files/model_arch.txt']

# Experiment

Now, it is time to finally run your ablations! Have fun!

In [24]:
# from pandas_profiling import ProfileReport
from ipywidgets import FloatProgress

In [25]:
# Iterate over number of epochs to train and evaluate your model
torch.cuda.empty_cache()

best_acc = 0.0 ### Monitor best accuracy in your run

for epoch in range(config['epochs']):
    print("\nEpoch {}/{}".format(epoch+1, config['epochs']))

    train_loss = train(model, optimizer, criterion, scheduler, train_loader)
    accuracy, val_loss = eval(model, val_loader)

    print("\tTrain Loss: {:.4f}".format(train_loss))
    print("\tValidation Accuracy: {:.2f}%".format(accuracy))


    ### Log metrics at each epoch in your run - Optionally, you can log at each batch inside train/eval functions (explore wandb documentation/wandb recitation)
    wandb.log({"train loss": train_loss, "validation accuracy": accuracy, "validation loss": val_loss})

    ### Save checkpoint if accuracy is better than your current best
    if accuracy >= best_acc:
        torch.save(model.state_dict(), 'exp_structure_v2/best-parameters_1.pt')
        best_acc = accuracy
        
        
    ### Save checkpoint with information you want
    torch.save({'epoch': epoch,
          'model_state_dict': model.state_dict(),
          'optimizer_state_dict': optimizer.state_dict(),
          'loss': train_loss,
          'acc': accuracy}, 
    './model_checkpoint.pth')

    ### Save checkpoint in wandb
    wandb.save('checkpoint.pth')


    # Is your training time very high? Look into mixed precision training if your GPU (Tesla T4, V100, etc) can make use of it 
    # Refer - https://pytorch.org/docs/stable/notes/amp_examples.html

### save best model parameters
model.load_state_dict(torch.load('exp_structure_v2/best-parameters_1.pt'))
### Finish your wandb run
run.finish()


Epoch 1/50


0it [00:00, ?it/s]

	Train Loss: 0.8517
	Validation Accuracy: 78.16%

Epoch 2/50


0it [00:00, ?it/s]

	Train Loss: 0.6885
	Validation Accuracy: 80.22%

Epoch 3/50


...

0it [00:00, ?it/s]

	Train Loss: 0.4385
	Validation Accuracy: 84.77%

Epoch 50/50


0it [00:00, ?it/s]

	Train Loss: 0.4385
	Validation Accuracy: 84.77%


train loss,█▅▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
validation accuracy,▁▃▄▅▅▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇████████████████████
validation loss,█▆▅▄▄▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train loss,0.43851
validation accuracy,84.76812
validation loss,0.46202


In [26]:
eval(model, val_loader)

(84.77421372740899, 0.4621569458729614)

# Testing and submission to Kaggle

Before we get to the following code, make sure to see the format of submission given in *random_submission.csv*. Once you have done so, it is time to fill the following function to complete your inference on test data. Refer the eval function from previous cells to get an idea of how to go about completing this function.

In [27]:
def test(model, test_loader):
    ### What you call for model to perform inference?
    model.eval()

    ### List to store predicted phonemes of test data
    test_predictions = []

    ### Which mode do you need to avoid gradients?
    with torch.inference_mode():

        for i, frames in enumerate(tqdm(test_loader)):

            frames = frames.float().to(device)             

            output = model(frames)

            ### Get most likely predicted phoneme with argmax
            predicted_phonemes = torch.argmax(output, dim=1)

            ### How do you store predicted_phonemes with test_predictions? Hint, look at eval 
            test_predictions.extend(predicted_phonemes.cpu().tolist())
          
    return test_predictions

In [28]:
predictions = test(model, test_loader)

  0%|          | 0/949 [00:00<?, ?it/s]

In [29]:
len(predictions)

1943253

In [30]:
### Create CSV file with predictions
with open("./exp_structure_v2/submission_largemodel_1.csv", "w+") as f:
    f.write("id,label\n")
    for i in range(len(predictions)):
        f.write("{},{}\n".format(i, predictions[i]))

In [31]:
result_df = pd.read_csv('./exp_structure_v2/submission_largemodel_1.csv')
result_df.shape

(1943253, 2)

In [32]:
### Submit to kaggle competition using kaggle API
# !kaggle competitions submit -c 11-785-f22-hw1p2 -f ./exp_scheduler/submission_CosineAnnealingLR_3.csv -m "Test Submission"